# Pandera: Validación de Esquemas para Pandas

Este notebook demuestra cómo usar Pandera para validar esquemas de DataFrames.

**Referencia:** [Pandera](../calidad/herramientas/pandera-validacion-pandas.md)

**Objetivos:**
- Definir esquemas de validación
- Validar DataFrames
- Integrar en pipelines

## 1. Instalación

In [ ]:
# Descomenta para instalar
# !pip install pandera

print("💡 Para instalar: pip install pandera")

## 2. Importar librerías

In [ ]:
import pandas as pd
import numpy as np

# Pandera
try:
    import pandera as pa
    from pandera import Column, Check
    print("✅ Pandera importado")
except ImportError:
    print("⚠️ Pandera no instalado. Ejecuta: pip install pandera")

## 3. Definir esquema

In [ ]:
try:
    # Definir esquema
    schema = pa.DataFrameSchema({
        "nombre": Column(str),
        "edad": Column(int, Check.ge(0), Check.le(120)),
        "email": Column(str, Check.str_matches(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')),
        "precio": Column(float, Check.gt(0)),
    })
    
    print("✅ Esquema definido")
    print("\nEsquema:")
    print("  - nombre: string")
    print("  - edad: int, entre 0 y 120")
    print("  - email: string, formato email válido")
    print("  - precio: float, mayor que 0")
    
except NameError:
    print("💡 Pandera no está instalado. Ejemplo de esquema:")
    print("""
    schema = pa.DataFrameSchema({
        "nombre": Column(str),
        "edad": Column(int, Check.between(0, 120)),
        "email": Column(str, Check.str_matches(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')),
        "precio": Column(float, Check.gt(0)),
    })
    """)

## 4. Validar DataFrame válido

In [ ]:
# Crear DataFrame válido
df_valido = pd.DataFrame({
    'nombre': ['Juan', 'María'],
    'edad': [28, 35],
    'email': ['juan@example.com', 'maria@example.com'],
    'precio': [10.5, 20.0]
})

print("DataFrame a validar:")
print(df_valido)

try:
    # Validar
    df_validado = schema.validate(df_valido)
    print("\n✅ Validación exitosa!")
    print(df_validado)
except NameError:
    print("\n💡 Pandera no instalado. El DataFrame es válido según el esquema.")

## 5. Validar DataFrame inválido

In [ ]:
# Crear DataFrame inválido
df_invalido = pd.DataFrame({
    'nombre': ['Juan', 'María'],
    'edad': [28, 150],  # 150 está fuera de rango
    'email': ['juan@example.com', 'email-invalido'],  # Email inválido
    'precio': [10.5, -5.0]  # Precio negativo
})

print("DataFrame a validar (con errores):")
print(df_invalido)

try:
    # Intentar validar (debería fallar)
    df_validado = schema.validate(df_invalido)
    print("\n✅ Validación exitosa (inesperado)")
except Exception as e:
    print(f"\n❌ Error de validación (esperado):")
    print(f"   {type(e).__name__}")
    print(f"   {str(e)[:200]}...")
except NameError:
    print("\n💡 Pandera no instalado. Este DataFrame fallaría la validación:")
    print("   - edad: 150 está fuera de rango (0-120)")
    print("   - email: 'email-invalido' no es un email válido")
    print("   - precio: -5.0 no es mayor que 0")

## 6. Integración en pipeline

In [ ]:
try:
    # Esquema de entrada
    schema_entrada = pa.DataFrameSchema({
        "fecha": Column("datetime64[ns]"),
        "producto_id": Column(int),
        "cantidad": Column(int, Check.gt(0)),
        "precio": Column(float, Check.gt(0)),
    })
    
    # Esquema de salida
    schema_salida = pa.DataFrameSchema({
        "fecha": Column("datetime64[ns]"),
        "producto_id": Column(int),
        "total": Column(float, Check.gt(0)),
    })
    
    def pipeline_con_validacion(df):
        # Validar entrada
        df = schema_entrada.validate(df)
        
        # Transformar
        df['total'] = df['cantidad'] * df['precio']
        df = df[['fecha', 'producto_id', 'total']]
        
        # Validar salida
        df = schema_salida.validate(df)
        
        return df
    
    # Ejemplo de uso
    df_ejemplo = pd.DataFrame({
        'fecha': pd.date_range('2024-01-01', periods=5, freq='D'),
        'producto_id': [1, 2, 3, 4, 5],
        'cantidad': [2, 3, 1, 4, 2],
        'precio': [10.0, 20.0, 15.0, 25.0, 30.0]
    })
    
    resultado = pipeline_con_validacion(df_ejemplo)
    print("✅ Pipeline con validación ejecutado")
    print(resultado)
    
except NameError:
    print("💡 Pandera no instalado. Ejemplo de pipeline con validación:")
    print("""
    def pipeline_con_validacion(df):
        # Validar entrada
        df = schema_entrada.validate(df)
        
        # Transformar
        df['total'] = df['cantidad'] * df['precio']
        
        # Validar salida
        df = schema_salida.validate(df)
        
        return df
    """)

---

**Próximo paso:** Lee [Pandera](../calidad/herramientas/pandera-validacion-pandas.md) para más detalles.